In [49]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
import keras_tuner as kt
import os

In [ ]:
housing = fetch_california_housing(as_frame=True)
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target, random_state=42, test_size=0.10
)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42
)

In [50]:
model = make_pipeline(RandomForestRegressor(random_state=1))

model.fit(X_train_full, y_train_full)

preds = model.predict(X_test)

mean_squared_error(y_test, preds, squared=False), r2_score(y_test, preds)


(0.5050142159466008, 0.8087600713546601)

In [ ]:
X_train

# Sequencial network for Housing prices(with Callbacks, TensorBoard and HyperParam tuning )

In [ ]:
tf.keras.backend.clear_session()
tf.random.set_seed(55)


# The function to define hyperparameter space and the model itself


def build_model(HyperParam):
    n_hidden = HyperParam.Int("n_hidden", min_value=1, max_value=4, default=2)
    n_neurons = HyperParam.Int("n_neurons", min_value=16, max_value=80)
    learning_rate = HyperParam.Choice("learning_rate", values=[0.01, 0.001, 0.0001])
    optimizer = HyperParam.Choice("optimizer", values=["sgd", "adam"])

    model = tf.keras.Sequential()
    norm_layer = tf.keras.layers.Normalization()
    norm_layer.adapt(X_train)
    model.add(norm_layer)

    if optimizer == "sgd":
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    else:
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    for _ in range(n_hidden):
        model.add(tf.keras.layers.Dense(n_neurons, activation="relu"))
    model.add(tf.keras.layers.Dense(1))

    model.compile(loss="mse", optimizer=optimizer, metrics=["RootMeanSquaredError"])

    return model

# RandomSearch with callbacks

In [ ]:
random_search = kt.RandomSearch(
    build_model,
    objective=kt.Objective("val_root_mean_squared_error", direction="min"),
    max_trials=20,
    overwrite=True,
    directory="my_housing",
    project_name="rand_search_1",
    seed=42,
)

root_logdir = os.path.join(random_search.project_dir, "tensorboard")
tensorboard_cb = tf.keras.callbacks.TensorBoard(root_logdir)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(
    patience=2, restore_best_weights=True
)
# Also here could be a callback to save weights after each n epoch
# ( if the training is long and computer crashes to restore the model)

random_search.search(
    X_train,
    y_train,
    epochs=50,
    validation_data=(X_valid, y_valid),
    callbacks=[early_stopping_cb, tensorboard_cb],
)

In [ ]:
# getting the best model itself can be done like this
top_models = random_search.get_best_models(num_models=3)
best_model = top_models[0]  # The best model

In [ ]:
# Getting top hyperparams , liek this
best_params = random_search.get_best_hyperparameters(num_trials=3)
best_params[0].values  # best hyperparams
# Or like this
best_trial = random_search.oracle.get_best_trials(num_trials=3)[0]
best_trial.summary()

In [ ]:
test_loss, test_rmse = best_model.evaluate(X_test, y_test)
test_rmse

# HyperBand search with Callbacks

In [ ]:
hyper_search = kt.Hyperband(
    build_model,
    objective=kt.Objective("val_root_mean_squared_error", direction="min"),
    max_epochs=10,
    overwrite=True,
    factor=3,
    directory="my_housing",
    project_name="hypeband_search_1",
    seed=42,
)


root_logdir = os.path.join(hyper_search.project_dir, "tensorboard")
tensorboard_cb = tf.keras.callbacks.TensorBoard(root_logdir)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(
    patience=2, restore_best_weights=True
)
# Also here could be a callback to save weights after each n epoch
# ( if the training is long and computer crashes to restore the model)

hyper_search.search(
    X_train,
    y_train,
    epochs=50,
    validation_data=(X_valid, y_valid),
    callbacks=[early_stopping_cb, tensorboard_cb],
)

In [43]:
top_models_2 = hyper_search.get_best_models(num_models=3)
best_model_2 = top_models_2[0]  # The best model

2023-08-30 16:40:40.057398: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-08-30 16:40:40.070375: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-08-30 16:40:41.210537: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-08-30 16:40:41.222469: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-08-30 16:40:42.625543: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-08-30 16:40:42.638195: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [ ]:
best_trial_2 = hyper_search.oracle.get_best_trials(num_trials=3)[0]
best_trial_2.summary()

In [ ]:
test_loss_2, test_rmse_2 = best_model_2.evaluate(X_test, y_test)
test_rmse_2

# The tuning of model.fit parameters sucg as batch size , or tuning of data preprocessing steps (such as Normalization) by subclassing the kt.HyperModel

In [ ]:
class MyModel(kt.HyperModel):
    def build(self, HyperParam):
        return build_model(
            HyperParam
        )  # The search of main HyperParam , such as n_layers, n_units, learning_rate etc.

    def fit(self, HyperParam, model, X, y, **kwargs):
        batch_size = HyperParam.Int("batch_size", min_value=12, max_value=256, default=32)
        return model.fit(X, y, batch_size=batch_size, **kwargs) # Search for best batch_size in addition to the rest


hyperband_tuner = kt.Hyperband(
    MyModel(),
    objective=kt.Objective("val_root_mean_squared_error", direction="min"),
    max_epochs=10,
    overwrite=True,
    factor=3,
    directory="my_housing",
    project_name="hypeband_search_with_batch_size",
    seed=42,
)

root_logdir = os.path.join(hyperband_tuner.project_dir, "tensorboard")
tensorboard_cb = tf.keras.callbacks.TensorBoard(root_logdir)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(
    patience=2, restore_best_weights=True
)

hyperband_tuner.search(X_train,
    y_train,
    epochs=50,
    validation_data=(X_valid, y_valid),
    callbacks=[early_stopping_cb, tensorboard_cb],
)

In [44]:
top_models_3 = hyperband_tuner.get_best_models(num_models=3)
best_model_3 = top_models_3[0]  # The best model

2023-08-30 16:40:43.906297: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-08-30 16:40:43.919689: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-08-30 16:40:45.210403: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-08-30 16:40:45.224331: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-08-30 16:40:46.353565: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-08-30 16:40:46.365313: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [45]:
best_trial_3 = hyperband_tuner.oracle.get_best_trials(num_trials=3)[0]
best_trial_3.summary()

Trial 0017 summary
Hyperparameters:
n_hidden: 4
n_neurons: 38
learning_rate: 0.001
optimizer: sgd
batch_size: 133
tuner/epochs: 10
tuner/initial_epoch: 4
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 0015
Score: 0.7174965739250183


In [46]:
test_loss_3, test_rmse_3 = best_model_3.evaluate(X_test, y_test)
test_rmse_3

18/65 [=======>......................] - ETA: 0s - loss: 0.5355 - root_mean_squared_error: 0.7318

2023-08-30 16:40:50.652118: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


65/65 [==============================] - 0s 5ms/step - loss: 0.5609 - root_mean_squared_error: 0.7490


0.7489539980888367

In [1]:
%tensorboard --logdir

UsageError: Line magic function `%tensorboard` not found.
